In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import LineString
import pandas as pd
import time

start_time = time.time() 

# Define file paths
toy_node_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\KOREARL_NODE.shp"
toy_edge_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\KOREARL_EDGE.shp"

# Load the shapefiles
toy_node = gpd.read_file(toy_node_path)
toy_edge = gpd.read_file(toy_edge_path)

toy_node = toy_node.to_crs(epsg=3857)
toy_edge = toy_edge.to_crs(epsg=3857)

print("📄 Node 속성 컬럼:")
print(toy_node.columns)
print("\n📄 Edge 속성 컬럼:")
print(toy_edge.columns)

print("\n🔍 toy_node 샘플:")
print(toy_node.head())
print("\n🔍 toy_edge 샘플:")
print(toy_edge.head())

# Create a plot with network and basemap
fig, ax = plt.subplots(figsize=(7, 7))
toy_edge.plot(ax=ax, color="grey", linewidth=1.8, label="Edges")

node_coords = toy_node.geometry.apply(lambda pt: (pt.x, pt.y)).tolist()
x_coords, y_coords = zip(*node_coords)
ax.scatter(x_coords, y_coords, color="black", s=2.3, label="Nodes", zorder=3, alpha=0.5)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zorder=0)
plt.legend()
plt.title("Scotland Railway Network")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

In [ ]:
import networkx as nx



# 1. Node numbering (n1, n2, ...)
round_coord = lambda coord: (round(coord[0], 6), round(coord[1], 6))

# 1. node_id → geometry
node_info_list = [
    (row.node_id, round_coord((row.geometry.x, row.geometry.y)))
    for row in toy_node.itertuples()
]

nodes = {}
node_meta = []

for node_id, coord in node_info_list:
    node_name = f"n{int(node_id)}"  # ✅ node_id 그대로 사용
    nodes[node_name] = coord
    node_meta.append({
        "node_id": int(node_id),
        "node_name": node_name,
        "geometry": coord
    })

# 좌표 → node_name 매핑
coord_to_node = {coord: node_name for node_name, coord in nodes.items()}

# 출력 확인
print("====== Node info (직접 매핑) ======")
for meta in node_meta:
    print(f"{meta['node_name']} (ID: {meta['node_id']}): {meta['geometry']}")



# 2. Edge numbering (e1, e2, ...)
nodeid_to_nodename = {meta["node_id"]: meta["node_name"] for meta in node_meta}
toy_edge['from_node_name'] = toy_edge['from_node_'].map(nodeid_to_nodename)
toy_edge['to_node_name'] = toy_edge['to_node_id'].map(nodeid_to_nodename)

edge_pairs = set()

for _, row in toy_edge.iterrows():
    from_node = row['from_node_name']
    to_node = row['to_node_name']
    sorted_pair = tuple(sorted([from_node, to_node]))
    edge_pairs.add(sorted_pair)

sorted_edge_pairs = sorted(edge_pairs, key=lambda x: (x[0], x[1]))
print(sorted_edge_pairs)

edges = {}
edge_number = 1

for from_node, to_node in sorted_edge_pairs:
    edges[f"e{edge_number}"] = (from_node, to_node)
    edges[f"e{edge_number + 1}"] = (to_node, from_node)
    edge_number += 2

edge_records = []
for edge_name, (from_node, to_node) in edges.items():
    match = toy_edge[((toy_edge['from_node_name'] == from_node) & (toy_edge['to_node_name'] == to_node)) |
                     ((toy_edge['from_node_name'] == to_node) & (toy_edge['to_node_name'] == from_node))]
    
    if not match.empty:
        journeys = match.iloc[0]['journeys']
        geometry = match.iloc[0]['geometry']
        edge_id = match.iloc[0]['edge_id']
    else:
        journeys = None
        geometry = None
        edge_id = None

    edge_records.append({
        'edge_id': edge_id,
        'edge_name': edge_name,
        'from_node_name': from_node,
        'to_node_name': to_node,
        'journeys': journeys,
        'geometry': geometry
    })

toy_edge_bidirectional = pd.DataFrame(edge_records)

toy_edge_bidirectional['edge_num'] = toy_edge_bidirectional['edge_name'].str.extract(r'e(\d+)').astype(int)
toy_edge_bidirectional = toy_edge_bidirectional.sort_values(by='edge_num').reset_index(drop=True)
toy_edge_bidirectional = toy_edge_bidirectional.drop(columns='edge_num')

print("\n====== Edge info ======")
pd.set_option('display.max_rows', None)
print(toy_edge_bidirectional)



# 3. Convert edges to arc format
arcs = [(u, v) for _, (u, v) in edges.items()]
#print("\nArcs:", arcs)



# 4. Compute Euclidean distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}
#print("Arc Distances:", arc_distance)



# 5. Create a network graph
# Create the graph using extracted data
G = nx.DiGraph()

# Add nodes and edges
for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
    G.add_edge(u, v, weight=arc_distance[edge_name])

# Remove duplicate edges
unique_edges = set()
edge_name_map = {}

for edge_name, (u, v) in edges.items():
    if (v, u) not in unique_edges:
        unique_edges.add((u, v))
        
        reverse_edge_name = [k for k, (a, b) in edges.items() if (a, b) == (v, u)]
        if reverse_edge_name:
            journeys = toy_edge_bidirectional[
                (toy_edge_bidirectional['from_node_name'] == u) &
                (toy_edge_bidirectional['to_node_name'] == v)
            ]['journeys'].values

            journeys_val = int(journeys[0]) if len(journeys) > 0 else "?"
            edge_label = f"{edge_name}, {reverse_edge_name[0]} ({journeys_val})"
        else:
            edge_label = edge_name

        edge_name_map[(u, v)] = edge_label


# Plot the network topology
plt.figure(figsize=(7, 7))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=250, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(unique_edges),  
    arrowstyle='-',
    min_target_margin=10,
    min_source_margin=10
)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_name_map, font_size=6)
nx.draw_networkx_labels(G, pos, font_size=7)
plt.title("Network with Unique Edge Labels")
plt.show()



In [ ]:
import sys
import os
import gc
import re
import json
import copy

# Scientific Libraries
import numpy as np
from scipy.stats import beta

# Optimization
from gurobipy import Model, GRB, quicksum

# Graph & Plotting\
import matplotlib
matplotlib.use("Agg")

import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl

# MBNpy Modules
import matplotlib
matplotlib.use("Agg")  # GUI 없는 환경에서도 작동하는 안전한 backend
from mbnpy import brc, cpm, variable, operation, branch, config

# Local Module
import batch

# Clean up memory
gc.collect()
%matplotlib inline


In [ ]:
nodes = nodes  
edges = edges  
arcs = arcs
arc_distance = arc_distance 



# 1. Generate arc failure probabilities based on arc distances (longer distance -> higher failure probability)
# Get the minimum and maximum arc distances
min_dist = min(arc_distance.values())
max_dist = max(arc_distance.values())

# Failure probability range (0.01 ~ 0.3)
min_prob = 0.01
max_prob = 0.3

def compute_failure_probability(distance, min_dist, max_dist, min_prob, max_prob):
    normalized_dist = (distance - min_dist) / (max_dist - min_dist) # Normalize to [0,1]
    return round(min_prob + normalized_dist * (max_prob - min_prob), 4)

# Generate failure probability for each edge
probs_dynamic = {
    edge: {
        0: compute_failure_probability(dist, min_dist, max_dist, min_prob, max_prob),
        1: round(1 - compute_failure_probability(dist, min_dist, max_dist, min_prob, max_prob), 4)
    }
    for edge, dist in arc_distance.items()
}

# Sort edges by numeric order
def numeric_sort(edge):
    return int(edge[1:])  # Extract numeric part from "eX"

probs_sorted = {k: probs_dynamic[k] for k in sorted(probs_dynamic, key=numeric_sort)}

# Print formatted probabilities
#print(json.dumps(probs_sorted, indent=4, separators=(",", ": ")))
probs = probs_sorted
probs_cpm = copy.deepcopy(probs)
#print(probs_cpm)



# 2. Assign capacities to each arcs
# Initial intact capacity (Unchanged from original)
intact_capacity = {}

for edge_name, (u, v) in edges.items():
    match = toy_edge[((toy_edge['from_node_name'] == u) & (toy_edge['to_node_name'] == v)) |
                     ((toy_edge['from_node_name'] == v) & (toy_edge['to_node_name'] == u))]

    if not match.empty:
        intact_capacity[edge_name] = match.iloc[0]['journeys']
    else:
        intact_capacity[edge_name] = None  

print("Intact Capacities:", intact_capacity)

# Function to generate random component states (0 or 1) based on failure probabilities
def generate_comps_st(probs):
    comps_st = {}

    for edge, prob in probs.items():
        if isinstance(prob, dict) and 0 in prob and 1 in prob:  # Ensure correct structure
            comps_st[edge] = np.random.choice([0, 1], p=[prob[0], prob[1]])
        else:
            print(f"Warning: Invalid probability format for edge {edge}: {prob}")  

    return comps_st

# Compute arc capacities
comps_st = generate_comps_st(probs_sorted)  



# 3. Compute maximum allowable distance
# Demand data
json_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\Korea\demand_data.json"
with open(json_path, "r", encoding="utf-8") as f:
    demand = json.load(f)

# Compute max allowable distance
avg_velo = 149  # Speed in km/h
max_distance = {}
commodity_name_map = {}  

for idx, item in enumerate(demand, start=1):
    origin = item["origin"]
    destination = item["destination"]
    commodity_key = f"{origin}->{destination}"
    commodity_name = f"k{idx}"  

    distance = item["distance"]
    max_allowable_time = (distance * 60) / avg_velo + 180
    max_distance[commodity_name] = max_allowable_time * avg_velo / 60

    commodity_name_map[commodity_name] = {
        "key": commodity_key,
        "origin": origin,
        "destination": destination
    }

    print(f"\nCommodity: {commodity_name}")
    print(f"  OD Pair: {commodity_key}")
    print(f"  Distance (from JSON): {distance:.2f} km")
    print(f"  Maximum allowable time: {max_allowable_time:.2f} minutes")
    print(f"  Maximum allowable distance: {max_distance[commodity_name]:.2f} km")

In [ ]:
# Build directed graph with weighted arcs
G = nx.DiGraph()
for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_id, (u, v) in edges.items():
    G.add_edge(u, v, weight=arc_distance[edge_id])

# Initialize demand for each arc
edge_demand = { (u, v): 0 for u, v in G.edges }

# Accumulate demand from all shortest paths
for info in demand:
    origin = info["origin_name"]         # use node name like 'n18'
    destination = info["destination_name"]
    amount = info["journeys"]            # amount is stored as 'journeys'

    try:
        paths = list(nx.all_shortest_paths(G, source=origin, target=destination, weight="weight"))
        for path in paths:
            for i in range(len(path) - 1):
                edge = (path[i], path[i + 1])
                if edge in edge_demand:
                    edge_demand[edge] += amount
                    print(f"🟦 OD {origin} → {destination}, amount={amount} ➜ edge {edge} += {amount} (total: {edge_demand[edge]})")
    except nx.NetworkXNoPath:
        print(f"⚠️ No path found for {origin} → {destination}")
        continue

# Log-scaled normalization
normalized_demand = {
    (u, v): np.log10(d + 1) for (u, v), d in edge_demand.items()
}
D_max = max(normalized_demand.values())
D_min = min(normalized_demand.values())

normalized_demand = {
    (u, v): (val - D_min) / (D_max - D_min)
    for (u, v), val in normalized_demand.items()
}

# Visualization
from matplotlib.colors import LinearSegmentedColormap
start_rgb = (245/255, 250/255, 254/255)    
mid_rgb   = (100/255, 169/255, 211/255)       
end_rgb   = (8/255, 50/255, 110/255)    
cmap = LinearSegmentedColormap.from_list("custom_3color_gradient", [start_rgb, mid_rgb, end_rgb])

pos = nx.get_node_attributes(G, "pos")
fig, ax = plt.subplots(figsize=(14, 14))

# Nodes
# 2. 라벨 시각화 (normalized demand 그리는 코드 이후에 삽입)

# 1. 엣지 라벨 정의 (값이 0보다 큰 경우만 표시)
edge_labels = {
    (u, v): f"{edge_demand[(u, v)]:.0f}"
    for (u, v) in G.edges
    if edge_demand[(u, v)] > 0
}

nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8,
    font_color="darkred",
    rotate=False,
    ax=ax
)


# Edges with normalized demand
for (u, v), value in normalized_demand.items():
    color = cmap(value)
    nx.draw_networkx_edges(
        G, pos, edgelist=[(u, v)],
        width=1.8,
        edge_color=[color],
        arrowstyle="->",
        arrowsize=8,
        alpha=1,
        ax=ax,
        connectionstyle="arc3,rad=0.07"
    )

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs="EPSG:3857", zorder=0)
ax.tick_params(labelbottom=True, labelleft=True)
ax.set_axis_on()
ax.ticklabel_format(useOffset=False)

# Colorbar
norm = mpl.colors.Normalize(vmin=D_min, vmax=D_max)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Edge Demand Intensity (normalized)")

# Grid
ax.grid(True, which='major', linestyle='--', linewidth=0.5, color='gray', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=14, direction='out')
ax.set_axisbelow(True)  

plt.title("OD Demands for Scotland Railway Network", fontsize=18)
plt.xlabel("Longitude", fontsize=14)
plt.ylabel("Latitude", fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def MCNF_od_systemfunc(arcs, comps_st, edges, arc_capacity, demand,max_distance, arc_distance, target_od):
    
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    # Construct a graph for shortest path calculations
    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1)) 

    # Create Gurobi model
    model = Model("Network Flow Optimization")
    model.setParam('OutputFlag', 0) 

    # Define decision variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective: minimize total unmet demand
    model.setObjective(
        quicksum(unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Distance constraints for each commodity
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        
        model.addConstr(distance_expr <= max_distance[k] * total_flow)

    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        origin = demand[target_od]['origin']
        destination = demand[target_od]['destination']
        amount = demand[target_od]['amount']

        # Sum of flow into destination
        flow_to_dest = sum(
            flow[target_od, i, destination].x
            for i, j in arcs
            if j == destination and (target_od, i, j) in flow
        )

        flow_ratio = flow_to_dest / amount if amount > 0 else 0
        if flow_ratio > 0.9:
            sys_st = 's'

            min_comps_st = {}
            for (k, i, j), var in flow.items():
                if k == target_od and var.x > 0:
                    link_name = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
                    if link_name:
                        min_comps_st[link_name] = 1

        else:
            sys_st = 'f'
            min_comps_st = None    
        
        print(f"Used Components: {min_comps_st}")

        return flow_ratio, sys_st, min_comps_st
    
    else:
        return None, None, None

In [ ]:
def MCNF_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum
    import networkx as nx

    # Create a graph for shortest path calculations
    G = nx.Graph()
    for e, (i, j) in edges.items():
        G.add_edge(i, j, weight=arc_distance.get(e, 1))  # Set edge weights based on distance

    # Create Gurobi optimization model
    model = Model("Network Flow Optimization")
    model.setParam('OutputFlag', 0) 

    # Define variables
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
            capacity = arc_capacity.get(arc_key, 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective function: Minimize expected loss
    model.setObjective(
        quicksum(unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        arc_key = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
        model.addConstr(quicksum(flow[k, i, j] for k in demand if (k, i, j) in flow) <= arc_capacity.get(arc_key, 0))

    # Constraint 3: Commodity별 max_distance 적용
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get(e, 0) * flow[k, i, j] for e, (i, j) in edges.items() if (k, i, j) in flow)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if (k, i, j) in flow and i == origin)
        
        model.addConstr(distance_expr <= max_distance[k] * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal

        total_demand = sum(info['amount'] for info in demand.values())
        expected_loss = max(0.0, min(expected_loss, total_demand))  # [0, total_demand] 범위로 보정

        if expected_loss < 127364:
            sys_st = 's'

            # We can infer an associated minimum survival rule in case of network connectivity.
            min_comps_st = {}

            # Extract used links from optimization result
            for (k, i, j), var in flow.items():
                if var.x > 0:  # If flow is greater than 0, the edge is used in the solution
                    link_name = next((e for e, v in edges.items() if v == (i, j) or v == (j, i)), None)
                    if link_name:
                        min_comps_st[link_name] = 1  # Store as {e1: 1, e2: 1}

        else:
            sys_st = 'f'
            min_comps_st = None  # No survival rule needed for failed state
        
        return expected_loss, sys_st, min_comps_st

    else:
        return None, None, None  

In [ ]:
import networkx as nx
from networkx.algorithms.flow import shortest_augmenting_path

def shortestpath_systemfunc(arcs, comps_st, edges, arc_capacity, demand, max_distance, arc_distance):

    G = nx.DiGraph()

    for e, (i, j) in edges.items():
        if comps_st.get(e, 0) > 0:
            cap = arc_capacity.get(e, 1)
            dist = arc_distance.get(e, 1)
            G.add_edge(i, j, capacity=cap, weight=dist, link_id=e)
            G.add_edge(j, i, capacity=cap, weight=dist, link_id=e)

    expected_loss = 0.0
    used_links_set = set()

    for k, info in demand.items():
        origin = info["origin"]
        destination = info["destination"]
        amount = info["amount"]
        max_dist = max_distance.get(k, float('inf'))

        if origin not in G.nodes or destination not in G.nodes:
            expected_loss += amount
            continue

        G_temp = G.copy()
        G_temp.add_edge(destination, 'sink', capacity=1)  # 가상 목적지

        try:
            flow_value, flow_dict = nx.maximum_flow(
                G_temp, origin, 'sink', capacity='capacity', flow_func=shortest_augmenting_path
            )
        except Exception as e:
            flow_value = 0

        if flow_value == 0:
            expected_loss += amount
            continue

        try:
            path = nx.shortest_path(G, source=origin, target=destination, weight='weight')
            path_length = nx.shortest_path_length(G, source=origin, target=destination, weight='weight')

            if path_length > max_dist:
                expected_loss += amount
            else:
                for u, v in zip(path[:-1], path[1:]):
                    edge_id = G[u][v]['link_id']
                    used_links_set.add(edge_id)
        except nx.NetworkXNoPath:
            expected_loss += amount

    total_demand = sum(info["amount"] for info in demand.values())
    expected_loss = max(0.0, min(expected_loss, total_demand))

    if expected_loss < 127364:
        sys_st = 's'
        min_comps_st = {e: 1 for e in used_links_set}
    else:
        sys_st = 'f'
        min_comps_st = None

    return expected_loss, sys_st, total_demand


In [ ]:
demand_dict = {}

for idx, item in enumerate(demand, start=1):
    key = f"k{idx}"
    demand_dict[key] = {
        "origin": item["origin_name"],
        "destination": item["destination_name"],
        "amount": item["journeys"],
        "distance": item["distance"]
    }

# 2. 모든 컴포넌트가 정상 상태
comps_st = {edge: 1 for edge in intact_capacity}

# 3. arc capacity 설정
arc_capacity = {edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity}

# 4. MCNF 실행
expected_loss, sys_st, total_demand = shortestpath_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity=arc_capacity,
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)

# 5. 결과 출력
print("✅ Expected Loss:", expected_loss)
print("📌 System State:", sys_st)
print("📊 Total Demand:", total_demand)


In [ ]:
sys_fun_mcnf = lambda comps_st: MCNF_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity={e: intact_capacity[e] * comps_st[e] for e in comps_st},
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)

sys_fun_shortest = lambda comps_st: shortestpath_systemfunc(
    arcs=arcs,
    comps_st=comps_st,
    edges=edges,
    arc_capacity={e: intact_capacity[e] * comps_st[e] for e in comps_st},
    demand=demand_dict,
    max_distance=max_distance,
    arc_distance=arc_distance
)


# Print input values
print("\n🔹 Input Values:")
print("Component States (comps_st):", comps_st)
print("Edges:", edges)
print("Arc Capacity:", arc_capacity)
print("Demand:", demand_dict)
print("Max Distance:", max_distance)
print("Arc Distance:", arc_distance)

# Run the function and capture outputs
expected_loss_mcnf, sys_st_mcnf, min_comps_st_mcnf = sys_fun_mcnf(comps_st)
expected_loss_shortest, sys_st_shortest, min_comps_st_shortest = sys_fun_shortest(comps_st)

# Print output values
print("\n🔹 Output Values:")
print("System State (MCNF):", sys_st_mcnf)
print("System State (Shortest):", sys_st_shortest)
print("Minimum component state (MCNF):", min_comps_st_mcnf) 
print("Minimum component state (Shortest):", min_comps_st_shortest) 

# Expected Loss Evaluation
By BRC algorithm

### Shortest path